In [9]:
import pandas as pd
import xlwt
import openpyxl
import numpy as np
import csv

In [10]:
# Path to your Excel file
original_excel_file = './August_2024/august.xls'

# Path to your new Excel file
new_excel_file = './August_2024/august_middle_1.xls'

In [11]:
df = pd.read_excel(original_excel_file, sheet_name=None)
all_sheets = pd.read_excel(original_excel_file, sheet_name=None)

In [12]:
# To check wheteher every sheet is read or not
# # Print columns for each sheet
# for sheet_name, df in all_sheets.items():
#     print(f"Sheet: {sheet_name}")

# # Print the first 5 rows for each sheet
# for sheet_name, df in all_sheets.items():
#     print(f"Sheet: {sheet_name}")

In [13]:
# Remove all the rows in the dataframe which contains np.nan in the column "Transcribed By"
for sheet_name, df in all_sheets.items():
    # Check if 'Transcribed By' column exists in the DataFrame
    if 'Transcribed By' in df.columns:
        df.dropna(subset=['Transcribed By'], inplace=True)
    else:
        print(f"Skipping sheet '{sheet_name}' due to missing 'Transcribed By' column.")

In [14]:
# Create a new empty set and store the first 2 characters of each unique value in the Transcribed By column from every sheet
unique_values_2 = set()

try: 
    for sheet_name, df in all_sheets.items():
        unique_values_2.update(df['Transcribed By'].str[:2].unique())
    print(unique_values_2)

except AttributeError:
    print(f"Error occured in {sheet_name}")

{'HS', 'PS', 'RM'}


In [15]:
columns = ["VoiceFile Name", "Document Name", "DOB", "Doctor", "Date on FTP", "Download Date", "DOS", "Transcribe Date", "Transcribed By", "Line Count"]

In [16]:
# Assuming all_sheets and unique_values_2 are defined
created_sheets = set()

with pd.ExcelWriter(new_excel_file, engine='openpyxl') as writer:
    for sheet_name, df in all_sheets.items():
        
        # Check if 'Transcribed By' column exists in the DataFrame
        if 'Transcribed By' in df.columns:
            for value in unique_values_2:
                # Ensure that 'Transcribed By' values are strings
                df['Transcribed By'] = df['Transcribed By'].astype(str)

                # Filter rows based on string values in 'Transcribed By' column
                filtered_rows = df[df['Transcribed By'].str.startswith(value)]
                sheet_name_to_create = f'{sheet_name}_{value}'
                filtered_rows.to_excel(writer, sheet_name=sheet_name_to_create, index=False)
                created_sheets.add(sheet_name_to_create)
        else:
            print(f"Skipping sheet '{sheet_name}' due to missing 'Transcribed By' column.")

# Create a new workbook for merging similar sheets
merged_excel_file = './August_2024/august_MT.xls'
with pd.ExcelWriter(merged_excel_file, engine='openpyxl') as writer:
    for suffix in set(sheet_name[-2:] for sheet_name in created_sheets):
        # Concatenate only the specified columns
        merged_df = pd.concat([pd.read_excel(new_excel_file, sheet_name=sheet_name)[columns] for sheet_name in created_sheets if sheet_name.endswith(suffix)])

        # Format date columns
        for date_column in ["DOB", "Date on FTP", "Download Date", "DOS", "Transcribe Date"]:
            merged_df[date_column] = pd.to_datetime(merged_df[date_column], errors='coerce').dt.strftime('%m/%d/%Y')

        # Calculate the total line count
        total_line_count = merged_df["Line Count"].sum()

        # Create a new DataFrame for the total line count
        total_row = pd.DataFrame({"Line Count": [total_line_count]})

        # Concatenate the total row with the merged DataFrame
        merged_df = pd.concat([merged_df, total_row])

        # Write the DataFrame to the Excel file
        merged_df.to_excel(writer, sheet_name=suffix, index=False, header=columns)

C:\Users\Mohammad Soban\AppData\Local\Temp\ipykernel_19608\4230829842.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([pd.read_excel(new_excel_file, sheet_name=sheet_name)[columns] for sheet_name in created_sheets if sheet_name.endswith(suffix)])
C:\Users\Mohammad Soban\AppData\Local\Temp\ipykernel_19608\4230829842.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([pd.read_excel(new_excel_file, sheet_name=sheet_name)[columns] for sheet_name in 

Subject: Appreciation for Your Support During My Hopsitalization
